In [ ]:
!pip install pinecone

In [ ]:
from google.cloud import storage
import json
import pandas as pd
from pinecone import Pinecone
import time
from google import genai
from google.genai import types
import base64
import time
from google.api_core import exceptions as g_exceptions

In [ ]:
# Initialize GCS client
client = storage.Client()
bucket_name = "finetune-data123"
bucket = client.bucket(bucket_name)

In [ ]:
filepath_court = "main_pipiline_data/court"
filepath_tribunals = "main_pipiline_data/tribunals/tribunals"

def get_gcs_file_paths(bucket, court="", year=""):
    paths = []

    # Iterate over all blobs in the bucket
    for blob in bucket.list_blobs():  # optionally use prefix to speed up
        if not blob.name.endswith('.xml'):
            continue
        # Apply filters
        if court and court not in blob.name:
            continue
        if year and year not in blob.name:
            continue
        paths.append(blob.name)

    return paths

all_files = get_gcs_file_paths(bucket)

In [ ]:
all_files = all_files[0:10001] #run1
#all_files = all_files[10000:20001] #run2
#all_files = all_files[20000:30001] #run3
#all_files = all_files[30000:40001] #run4
#all_files = all_files[40000:50001] #run5
#all_files = all_files[50000:60001]
#all_files = all_files[60000:]
print(len(all_files))

10001


In [ ]:
import re
import json
from bs4 import BeautifulSoup
from google.cloud import storage

class FileProcessor:
    def __init__(self, bucket_name):
        self.client = storage.Client()
        self.bucket = self.client.bucket(bucket_name)

    def clean_text(self, text: str) -> str:
        text = re.sub(r'\n{2,}', '\n', text)
        text = re.sub(r' +', ' ', text)
        text = text.strip()
        # remove any line that begins with #judgment
        text = re.sub(r'(?m)^#judgment.*\n?', '', text)
        return text

    def get_metadata(self, soup, path):
      metadata = {"path": path}

      # FRBRWork metadata
      frbrwork = soup.find('frbrwork')
      if frbrwork:
          frbr_tags = {
              'frbruri': 'uri',
              'frbrdate': 'judgment_date',
              'frbrname': 'name',
              'frbrauthor': 'author',
              'frbrcountry': 'country',
              'frbrnumber': 'number',
              'frbrthis': 'this'
          }
          for tag_name, key in frbr_tags.items():
              tag = frbrwork.find(tag_name)
              if tag:
                  # Some have 'value', some have 'date', some have 'href'
                  value = tag.attrs.get('value') or tag.attrs.get('date') or tag.attrs.get('href')
                  if value:
                      metadata[key] = value

      # Capture proprietary UK metadata (namespaced)
      uk_tags = ['uk:court', 'uk:year', 'uk:number', 'uk:cite']
      for tag_name in uk_tags:
          tag = soup.find(tag_name)
          if tag:
              metadata[tag_name.replace('uk:', '')] = tag.get_text(strip=True)

      # Capture judges from references
      judges = [p.get('showas') for p in soup.find_all('tlcperson')]
      if judges:
          metadata['judges'] = judges

      return metadata


    def process_files(self, filepaths):
        """
        filepaths: list of paths in the bucket
        Returns a list of dicts: [{'metadata': ..., 'text': ...}, ...]
        """
        results = []
        i = 1
        for path in filepaths:
            i+=1
            print(f"{i}/{len(filepaths)}")
            blob = self.bucket.blob(path)
            # Download as string
            xml_content = blob.download_as_text()  # memory-efficient for one file at a time
            soup = BeautifulSoup(xml_content, "lxml")  # parse XML
            metadata = self.get_metadata(soup, path)

            # Get main text (adjust selector depending on your XML structure)
            text_tag = soup.find('Text') or soup  # fallback to whole document
            text = text_tag.get_text() if text_tag else ''
            clean = self.clean_text(text)

            results.append({
                'metadata': metadata,
                'text': clean
            })

        return results


In [ ]:
bucket_name = "finetune-data123"
filepaths = all_files

processor = FileProcessor(bucket_name)
processed_files = processor.process_files(filepaths)
# save results in a results file ni fietune-dataq


Streaming output truncated to the last 5000 lines.
5003/10001
5004/10001
5005/10001
5006/10001
5007/10001
5008/10001
5009/10001
5010/10001
5011/10001
5012/10001
5013/10001
5014/10001
5015/10001
5016/10001
5017/10001
5018/10001
5019/10001
5020/10001
5021/10001
5022/10001
5023/10001
5024/10001
5025/10001
5026/10001
5027/10001
5028/10001
5029/10001
5030/10001
5031/10001
5032/10001
5033/10001
5034/10001
5035/10001
5036/10001
5037/10001
5038/10001
5039/10001
5040/10001
5041/10001
5042/10001
5043/10001
5044/10001
5045/10001
5046/10001
5047/10001
5048/10001
5049/10001
5050/10001
5051/10001
5052/10001
5053/10001
5054/10001
5055/10001
5056/10001
5057/10001
5058/10001
5059/10001
5060/10001
5061/10001
5062/10001
5063/10001
5064/10001
5065/10001
5066/10001
5067/10001
5068/10001
5069/10001
5070/10001
5071/10001
5072/10001
5073/10001
5074/10001
5075/10001
5076/10001
5077/10001
5078/10001
5079/10001
5080/10001
5081/10001
5082/10001
5083/10001
5084/10001
5085/10001
5086/10001
5087/10001
5088/10001
508

In [ ]:
print(len(processed_files))

10001


In [ ]:
from pprint import pprint

pprint(processed_files[0]["metadata"])


{'author': '#ewca-civil',
 'cite': '[2001] EWCA Civ 540',
 'country': 'GB-UKM',
 'court': 'EWCA-Civil',
 'judges': ['LORD JUSTICE WARD', 'LORD JUSTICE MAY', 'LORD JUSTICE RIX'],
 'judgment_date': '2001-04-11',
 'name': 'G, Re Application for Judicial Review',
 'number': '540',
 'path': 'main_pipiline_data/court/ewca/ewca%2Fciv/2001/540.xml',
 'this': 'https://caselaw.nationalarchives.gov.uk/id/ewca/civ/2001/540',
 'uri': 'https://caselaw.nationalarchives.gov.uk/id/ewca/civ/2001/540',
 'year': '2001'}


In [ ]:
from google import genai
from google.genai import types
import base64
import time
from google.api_core import exceptions as g_exceptions

def generate_finetuned(text, retries=3, backoff=2):
    client = genai.Client(
        vertexai=True,
        project="651136027056",
        location="europe-west9",
    )

    model = "projects/651136027056/locations/europe-west9/endpoints/3743955839833079808"

    contents = [
        types.Content(
            role="user",
            parts=[types.Part.from_text(text=text)]
        )
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature=0.0,
        top_p=0.95,
        max_output_tokens=5000,
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
        thinking_config=types.ThinkingConfig(thinking_budget=0),
    )

    attempt = 0
    while attempt < retries:
        try:
            output_text = ""
            for chunk in client.models.generate_content_stream(
                model=model,
                contents=contents,
                config=generate_content_config,
            ):
                if chunk and getattr(chunk, "text", None):
                    output_text += chunk.text

            if not output_text.strip():
                raise ValueError("Empty response from model")

            return output_text  # ✅ Success, exit function

        except (g_exceptions.GoogleAPICallError, g_exceptions.RetryError,
                ValueError, Exception) as e:
            attempt += 1
            print(f"[Error] Attempt {attempt}/{retries} failed: {e}")

            if attempt < retries:
                sleep_time = backoff ** attempt
                print(f"Retrying in {sleep_time}s...")
                time.sleep(sleep_time)
            else:
                print("[Error] Max retries exceeded.")
                return "[Model error: no response]"


In [ ]:
def upload_to_pinecone(data):
    path = "data/chunked/chunked_file2.json"
    pc = Pinecone(api_key="xx")
    index = pc.Index(host="xx")

    # Process in chunks
    chunk_size = 96  # Adjust based on your memory constraints
    records = []
    rate_limit_delay = 60
    for i in range(0, len(data), chunk_size):
      chunk = data[i:i + chunk_size]
      try:
          index.upsert_records("summary", chunk)
          time.sleep(1)
          print(f"Uploaded records {i} to {i+len(chunk)}")
      except Exception as e:
          if "RESOURCE_EXHAUSTED" in str(e):
              print(f"Rate limit hit, waiting {rate_limit_delay} seconds...")
              time.sleep(rate_limit_delay)
              # Retry the same chunk
              i -= chunk_size
          else:
              print(e)


In [ ]:
import re


def split_into_sentence_chunks(text, max_bytes=30000):
    """
    Splits text into chunks ending at sentence boundaries,
    ensuring each chunk is <= max_bytes when UTF-8 encoded.
    """
    # Split text into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # Check byte length if we add this sentence
        if len((current_chunk + sentence + " ").encode('utf-8')) <= max_bytes:
            current_chunk += sentence + " "
        else:
            # Current chunk reached max_bytes, save it
            if current_chunk:  # avoid empty chunk
                chunks.append(current_chunk.strip())
            # Start new chunk with the current sentence
            current_chunk = sentence + " "

    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks



data_ready = []

for file1 in processed_files:
  metadata = file1["metadata"]
  query = "Summarize the following case file, while keeping the legal language. Be consise and include key information about the case such as scenario, precendent and outcome. Be concise: \n" + file1["text"]
  generated_text = generate_finetuned(query)
  #upload to bucket
  blob_name = f"summaries/{metadata['uri'].replace('/', '_')}.txt"
  blob = bucket.blob(blob_name)
  blob.upload_from_string(generated_text, content_type="text/plain")
  print("uploaded: "+ blob_name)
  text_bytes = generated_text.encode('utf-8')
  if len(text_bytes) <= 40000:
      record = {
            "_id": f"{metadata['uri']}",
            "text": chunk_text,
            "cite": metadata.get("cite",""),
            "file_path": metadata.get("path",""),
            "judgment_date": metadata.get("judgment_date",""),
            "name": metadata.get("name",""),
            "author": metadata.get("author",""),
            "judges": metadata.get("judges",""),
            "uri": metadata.get("uri"),
          }
      print("uploaded record")
      data_ready.append(record)
  else:
      print("splitting summary")
      sentence_chunks = split_into_sentence_chunks(generated_text)
      print(str(len(sentence_chunks)) + " Chunks created")
      for chunkid, chunk_text in enumerate(sentence_chunks):
          record = {
            "_id": f"{metadata['uri']}#{chunkid}",
            "text": chunk_text,
            "cite": metadata.get("cite",""),
            "file_path": metadata.get("path",""),
            "judgment_date": metadata.get("judgment_date",""),
            "name": metadata.get("name",""),
            "author": metadata.get("author",""),
            "judges": metadata.get("judges",""),
            "uri": metadata.get("uri"),
          }
          data_ready.append(record)



Streaming output truncated to the last 5000 lines.
uploaded: summaries/https:__caselaw.nationalarchives.gov.uk_id_ewca_civ_2004_1771.txt
uploaded record
uploaded: summaries/https:__caselaw.nationalarchives.gov.uk_id_ewca_civ_2004_1784.txt
uploaded record
uploaded: summaries/https:__caselaw.nationalarchives.gov.uk_id_ewca_civ_2004_1790.txt
uploaded record
uploaded: summaries/https:__caselaw.nationalarchives.gov.uk_id_ewca_civ_2004_18.txt
uploaded record
uploaded: summaries/https:__caselaw.nationalarchives.gov.uk_id_ewca_civ_2004_180.txt
uploaded record
uploaded: summaries/https:__caselaw.nationalarchives.gov.uk_id_ewca_civ_2004_1800.txt
uploaded record
uploaded: summaries/https:__caselaw.nationalarchives.gov.uk_id_ewca_civ_2004_1810.txt
uploaded record
uploaded: summaries/https:__caselaw.nationalarchives.gov.uk_id_ewca_civ_2004_1811.txt
uploaded record
uploaded: summaries/https:__caselaw.nationalarchives.gov.uk_id_ewca_civ_2004_1813.txt
uploaded record
uploaded: summaries/https:__casela

In [ ]:
import json
from google.cloud import storage

def save_json_to_bucket(bucket_name, blob_name, data):
    """
    Saves a Python object as a JSON file to a GCS bucket.

    Args:
        bucket_name (str): Name of your GCS bucket.
        blob_name (str): Path/filename in the bucket (e.g., 'data/chunked_file.json').
        data (list/dict): Python object to save as JSON.
    """
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Convert to JSON string
    json_str = json.dumps(data, indent=2)

    # Upload to GCS
    blob.upload_from_string(json_str, content_type='application/json')
    print(f"Saved JSON to gs://{bucket_name}/{blob_name}")

bucket_name = "finetune-data123"
blob_name = "main_pipiline_data/data_ready.json"

save_json_to_bucket(bucket_name, blob_name, data_ready)


In [ ]:
upload_to_pinecone(data_ready)